In [113]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Libraries for data preparation and model building
import statsmodels.graphics.api as sga
import statsmodels.formula.api as sfa
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# print multiple outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Setting global constants to ensure notebook results are reproducible
# PARAMETER_CONSTANT = ###

## Rationale to selecting predictor variables

In [199]:
df1 = pd.read_csv("df_train.csv")
print(f"There are {df1.shape[0]} rows and {df1.shape[1]} columns before")

# Remove unnecessary column(s)

df_train = df1.drop(labels="Unnamed: 0", axis=1)
print(f"There are {df_train.shape[0]} rows and {df_train.shape[1]} columns after")
df_train.head(1)
print('', end="\n\n")

There are 8763 rows and 49 columns before
There are 8763 rows and 48 columns after


,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.0,0.0,1.0,0.0,...,265.938,281.013,269.338615,269.338615,281.013,269.338615,274.254667,265.938,265.938,6715.666667


In [200]:
# Function to describe variable (including mode and median)

def describe(df):
    d = {0:[df.mean(), df.median(), df.mode()[0]]}
    dat = pd.DataFrame(data=d).rename(index={0: "Mean", 1: "Median", 2: "Mode"})
    return pd.concat([df.describe(), dat])

# Deal with null containing column(s)

df_train_clean = df_train.copy()
df_train_clean["Valencia_pressure"] = df_train_clean["Valencia_pressure"].fillna(df_train_clean["Valencia_pressure"].mode()[0])
print('', end="\n\n")

# Convert object dtypes to float

df_train_clean["Valencia_wind_deg"] = df_train_clean["Valencia_wind_deg"].str.extract("(\d+)").astype(int)
df_train_clean["Seville_pressure"] = df_train_clean["Seville_pressure"].str.extract("(\d+)").astype(int)
df_train_clean["time"] = pd.to_datetime(df_train_clean["time"])

# extract features from date

df_train_clean["time_year"] = df_train_clean["time"].dt.year.astype(int)
df_train_clean["time_month"] = df_train_clean["time"].dt.month.astype(int)
df_train_clean["time_day"] = df_train_clean["time"].dt.day.astype(int)
df_train_clean['time_dayofyear'] = df_train_clean['time'].dt.dayofyear.astype(int)
df_train_clean["time_hour"] = df_train_clean["time"].dt.hour.astype(int)
df_train_clean["time_weekday"] = df_train_clean["time"].dt.weekday.astype(int) # Monday is 0 and Sunday is 6
df_train_clean["time_weeknumber"] = df_train_clean["time"].dt.week.astype(int)

# Sort columns and drop noise ("time")

df_train_clean_sort = df_train_clean[sorted(df_train_clean)]
df_train_clean_sort = df_train_clean_sort.drop(labels="time", axis=1)
df_train_clean_sort.head(1)

,Barcelona_pressure,Barcelona_rain_1h,Barcelona_rain_3h,Barcelona_temp,Barcelona_temp_max,Barcelona_temp_min,Barcelona_weather_id,Barcelona_wind_deg,Barcelona_wind_speed,Bilbao_clouds_all,...,Valencia_wind_deg,Valencia_wind_speed,load_shortfall_3h,time_day,time_dayofyear,time_hour,time_month,time_weekday,time_weeknumber,time_year
0,1036.333333,0.0,0.0,281.013,281.013,281.013,800.0,42.666667,6.333333,0.0,...,5,0.666667,6715.666667,1,1,3,1,3,1,2015


In [201]:
# Variable selection

x_temp_min = x_0.filter(regex="min$", axis=1)
x_temp_min["Mean_temp_min"] = x_temp_min.mean(axis=1)
x_temp_min.head(1)

x_wind_deg1 = x_0.filter(regex=r'(Barcelona_wind_deg|Bilbao_wind_deg)', axis=1)
x_wind_deg1["Mean_wind_deg1"] = x_wind_deg1.mean(axis=1)
x_wind_deg1.head(1)

x_humidity = x_0.filter(regex="humidity", axis=1)
x_humidity["Mean_humidity"] = x_humidity.mean(axis=1)
x_humidity.head(1)

x_clouds_all = x_0.filter(regex="clouds", axis=1)
x_clouds_all["Mean_clouds_all"] = x_clouds_all.mean(axis=1)
x_clouds_all.head(1)

x_weather_id = x_0.filter(regex="weather", axis=1)
x_weather_id["Mean_weather_id"] = x_weather_id.mean(axis=1)
x_weather_id.head(1)

x_time = x_0.filter(regex="time", axis=1)
x_time["time_hourofyear"] = x_time["time_hour"] + ((x_time["time_dayofyear"] - 1) * 24)
x_time.tail(1)

,Barcelona_temp_min,Bilbao_temp_min,Madrid_temp_min,Seville_temp_min,Valencia_temp_min,Mean_temp_min
0,281.013,269.338615,265.938,274.254667,269.888,272.086456


,Barcelona_wind_deg,Bilbao_wind_deg,Mean_wind_deg1
0,42.666667,223.333333,133.0


,Madrid_humidity,Seville_humidity,Valencia_humidity,Mean_humidity
0,64.0,74.333333,75.666667,71.333333


,Bilbao_clouds_all,Madrid_clouds_all,Seville_clouds_all,Mean_clouds_all
0,0.0,0.0,0.0,0.0


,Barcelona_weather_id,Bilbao_weather_id,Madrid_weather_id,Seville_weather_id,Mean_weather_id
0,800.0,800.0,800.0,800.0,800.0


,time_day,time_dayofyear,time_hour,time_month,time_weekday,time_weeknumber,time_year,time_hourofyear
8762,31,365,21,12,6,52,2017,8757


In [202]:
x_select = x_time[["time_hourofyear"]].join(other = [x_temp_min["Mean_temp_min"], x_wind_deg1["Mean_wind_deg1"],
                                                         x_humidity["Mean_humidity"], x_clouds_all["Mean_clouds_all"],
                                                         x_weather_id["Mean_weather_id"]])
x_select.head(1)
x_select.shape

,time_hourofyear,Mean_temp_min,Mean_wind_deg1,Mean_humidity,Mean_clouds_all,Mean_weather_id
0,3,272.086456,133.0,71.333333,0.0,800.0


(8763, 6)

## Transform data function

In [114]:
def transform_alltime(csv):
    a = pd.read_csv(csv)
    
    # Convert object dtypes to float
    a["time"] = pd.to_datetime(a["time"])
    
    # extract features from date
    a["time_year"] = a["time"].dt.year.astype(int)
    a["time_month"] = a["time"].dt.month.astype(int)
    a["time_day"] = a["time"].dt.day.astype(int)
    a['time_dayofyear'] = a['time'].dt.dayofyear.astype(int)
    a["time_hour"] = a["time"].dt.hour.astype(int)
    a["time_weekday"] = a["time"].dt.weekday.astype(int) # Monday is 0 and Sunday is 6
    a["time_weeknumber"] = a["time"].dt.week.astype(int)
    
    # Sort columns and drop noise ("time")
    b = a[sorted(a)]
    b = b.drop(labels="time", axis=1)
    
    # Mean of variables
    c = b.filter(regex="min$", axis=1)
    c["Mean_temp_min"] = c.mean(axis=1)
    
    d = b.filter(regex=r'(Barcelona_wind_deg|Bilbao_wind_deg)', axis=1)
    d["Mean_wind_deg1"] = d.mean(axis=1)
    
    e = b.filter(regex="humidity", axis=1)
    e["Mean_humidity"] = e.mean(axis=1)
    
    f = b.filter(regex="clouds", axis=1)
    f["Mean_clouds_all"] = f.mean(axis=1)
    
    g = b.filter(regex="weather", axis=1)
    g["Mean_weather_id"] = g.mean(axis=1)
    
    h = b.filter(regex="time", axis=1)
    h["time_hourofyear"] = h["time_hour"] + ((h["time_dayofyear"] - 1) * 24)
    
    final = h.join(other = [c["Mean_temp_min"], d["Mean_wind_deg1"],
                                                 e["Mean_humidity"], f["Mean_clouds_all"],
                                                 g["Mean_weather_id"]])
    
    return final

def transform_HofY(csv):
    a = pd.read_csv(csv)
    
    # Convert object dtypes to float
    a["time"] = pd.to_datetime(a["time"])
    
    # extract features from date
    a["time_year"] = a["time"].dt.year.astype(int)
    a["time_month"] = a["time"].dt.month.astype(int)
    a["time_day"] = a["time"].dt.day.astype(int)
    a['time_dayofyear'] = a['time'].dt.dayofyear.astype(int)
    a["time_hour"] = a["time"].dt.hour.astype(int)
    a["time_weekday"] = a["time"].dt.weekday.astype(int) # Monday is 0 and Sunday is 6
    a["time_weeknumber"] = a["time"].dt.week.astype(int)
    
    # Sort columns and drop noise ("time")
    b = a[sorted(a)]
    b = b.drop(labels="time", axis=1)
    
    # Mean of variables
    c = b.filter(regex="min$", axis=1)
    c["Mean_temp_min"] = c.mean(axis=1)
    
    d = b.filter(regex=r'(Barcelona_wind_deg|Bilbao_wind_deg)', axis=1)
    d["Mean_wind_deg1"] = d.mean(axis=1)
    
    e = b.filter(regex="humidity", axis=1)
    e["Mean_humidity"] = e.mean(axis=1)
    
    f = b.filter(regex="clouds", axis=1)
    f["Mean_clouds_all"] = f.mean(axis=1)
    
    g = b.filter(regex="weather", axis=1)
    g["Mean_weather_id"] = g.mean(axis=1)
    
    h = b.filter(regex="time", axis=1)
    h["time_hourofyear"] = h["time_hour"] + ((h["time_dayofyear"] - 1) * 24)
    
    final = h[["time_hourofyear"]].join(other = [c["Mean_temp_min"], d["Mean_wind_deg1"],
                                                 e["Mean_humidity"], f["Mean_clouds_all"],
                                                 g["Mean_weather_id"]])
    
    return final

## Functions

In [52]:
# Normalize Data
def Normalize(df):
    scaler = MinMaxScaler()
    x_scaled = scaler.fit_transform(df)
    x_normalize = pd.DataFrame(x_scaled, columns=df.columns)
    return x_normalize

In [35]:
# Fit model
def fit_model(df, y=y_0):
    df_fit = df.copy()
    y_name = ''.join([col for col in y.columns])
    X_name = [col for col in df_fit.columns]

    # Build OLS formula string " y ~ X "

    formula_str = y_name+" ~ "+" + ".join(X_name)

    model = sfa.ols(formula=formula_str, data=df_fit.join(y))
    fitted = model.fit()
    print(fitted.summary())

In [53]:
# Standardize
def standardize(df):
    # Create scaler object
    scaler = StandardScaler()

    # Create scaled version of the predictors (there is no need to scale the response)
    X_scaled = scaler.fit_transform(df)

    # Convert the scaled predictor values into a dataframe
    X_standardise = pd.DataFrame(X_scaled,columns=df.columns)
    return X_standardise

In [187]:
# Train data using Ridge
def ridge_train(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x,
                                                        y["load_shortfall_3h"], 
                                                        test_size=0.2, 
                                                        shuffle=False)
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    b0 = float(ridge.intercept_)
    coeff = pd.DataFrame(ridge.coef_, x_select.columns, columns=['Coefficient'])

    print("Intercept:", float(b0))
    return coeff

In [188]:
# Fit a basic linear model
def model_accuracy(x, y, choice=LinearRegression()):
    X_train, X_test, y_train, y_test = train_test_split(x,
                                                        y["load_shortfall_3h"], 
                                                        test_size=0.2, 
                                                        shuffle=False)
    model_object = choice
    model_object.fit(X_train, y_train)
    train_pred = model_object.predict(X_train)
    test_pred = model_object.predict(X_test)
    
    print('Training RMSE:', np.sqrt(metrics.mean_squared_error(y_train, train_pred)))
    print('Model is, on average,', round(np.sqrt(metrics.mean_squared_error(y_train, train_pred))),'off prediction for training data')
    
    print('', end="\n")
    print('Training R_squared:', r2_score(y_train, train_pred))
    print('Model is good', "{:.0%}".format(r2_score(y_train, train_pred)), 'of the time')

    print('', end="\n")
    print('Testing RMSE:', np.sqrt(metrics.mean_squared_error(y_test, test_pred)))
    print('Model is , on average,', round(np.sqrt(metrics.mean_squared_error(y_test, test_pred))),'off prediction for testing data')
    
    print('', end="\n")
    print('Testing R_squared:', r2_score(y_test, test_pred))
    print('Model is good', "{:.0%}".format(r2_score(y_test, test_pred)), 'of the time')

In [156]:
# Predict Unseen
def predict_unseen(x, y, df_unseen, choice=LinearRegression()):
    X_train = x
    y_train = y["load_shortfall_3h"]
    X_test = df_unseen
    model_object = choice
    model_object.fit(X_train, y_train)
    predictions = model_object.predict(df_unseen)
    return predictions

In [66]:
# Save submission
def save_sub(series, time="df_test.csv", file="sample_submission_load_shortfall.csv"):
    my_series = pd.Series(series)
    load_shortfall_3h = my_series.to_frame()
    load_shortfall_3h = load_shortfall_3h.rename(columns = {0:'load_shortfall_3h'})
    time = pd.read_csv(time)[["time"]]
    sample = time.join(other = load_shortfall_3h)
    sample.to_csv(file, index=False)

## Work

In [70]:
y_0 = pd.read_csv("df_train.csv")[["load_shortfall_3h"]]

y_0.head(1)

,load_shortfall_3h
0,6715.666667


In [102]:
x_seen = transform_HofY("df_train.csv")
x_seen.head(1)
x_seen.shape

,time_hourofyear,Mean_temp_min,Mean_wind_deg1,Mean_humidity,Mean_clouds_all,Mean_weather_id
0,3,272.086456,133.0,71.333333,0.0,800.0


(8763, 6)

In [101]:
x_unseen = transform_HofY("df_test.csv")
x_unseen.head(1)
x_unseen.shape

,time_hourofyear,Mean_temp_min,Mean_wind_deg1,Mean_humidity,Mean_clouds_all,Mean_weather_id
0,0,282.55,185.0,68.222222,6.666667,800.25


(2920, 6)

In [204]:
Normalize(x_seen).head(1)

,time_hourofyear,Mean_temp_min,Mean_wind_deg1,Mean_humidity,Mean_clouds_all,Mean_weather_id
0,0.000342,0.02766,0.387191,0.661842,0.0,0.988095


In [205]:
standardize(x_seen).head(1)

,time_hourofyear,Mean_temp_min,Mean_wind_deg1,Mean_humidity,Mean_clouds_all,Mean_weather_id
0,-1.731947,-2.340125,-0.621239,0.513853,-1.302834,0.744832


In [206]:
ridge_train(x_seen, y_0)

Intercept: -23143.27963783583


,Coefficient
time_hourofyear,0.237741
Mean_temp_min,98.086223
Mean_wind_deg1,-10.429098
Mean_humidity,18.171724
Mean_clouds_all,-7.109671
Mean_weather_id,6.626397


In [203]:
model_accuracy(x_seen, y_0, choice=RandomForestRegressor())

Training RMSE: 1399.6761063180345
Model is, on average, 1400 off prediction for training data

Training R_squared: 0.9263018267182026
Model is good 93% of the time

Testing RMSE: 4807.425482959887
Model is , on average, 4807 off prediction for testing data

Testing R_squared: 0.028202951865606285
Model is good 3% of the time


In [153]:
# Get predictions of unseen data

# series_L = predict_unseen(x=x_seen, y=y_0, df_unseen=x_unseen, choice=LinearRegression())
# series_R = predict_unseen(x=x_seen, y=y_0, df_unseen=x_unseen, choice=Ridge())
# series_Lasso = predict_unseen(x=x_seen, y=y_0, df_unseen=x_unseen, choice=Lasso())
# series_RF = predict_unseen(x=x_seen, y=y_0, df_unseen=x_unseen, choice=RandomForestRegressor(random_state=137))
# series_RFpar = predict_unseen(x=x_seen, y=y_0, df_unseen=x_unseen,
#                               choice=RandomForestRegressor(n_estimators=100, max_depth=5, random_state=137))

**array([ 9839.23550254,  9874.22328293, 10176.95654912, ...,13713.97623095, 14099.32865803, 13610.03186923])**

In [154]:
# View predictions of unseen data 

# series_L
# series_R
# series_Lasso
# series_RF
# series_RFpar

In [208]:
# Save in submission format

# save_sub(series=series_RFpar, time="df_test.csv", file="sub_RFpar_HofY_5topmean.csv")